In [1]:
from urllib import urlretrieve
import os
import sys
import tarfile
from scipy import ndimage

In [3]:
url = 'http://yaroslavvb.com/upload/notMNIST/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print 'Found and verified', filename
  else:
    raise Exception(
      'Failed to verify' + filename + '. Can you get to it with a browser?')
  return filename

train_filename = maybe_download('notMNIST_large.tar.gz', 247336696)
test_filename = maybe_download('notMNIST_small.tar.gz', 8458043)

Found and verified notMNIST_large.tar.gz
Found and verified notMNIST_small.tar.gz


In [6]:
print train_filename
print test_filename


notMNIST_large.tar.gz
notMNIST_small.tar.gz


In [5]:
num_classes = 10

def extract(filename):
  tar = tarfile.open(filename)
  root = os.path.splitext(os.path.splitext(filename)[0])[0]  # remove .tar.gz
  print('Extracting data for %s. This may take a while. Please wait.' % root)
  sys.stdout.flush()
  tar.extractall()
  tar.close()
  data_folders = [
    os.path.join(root, d) for d in sorted(os.listdir(root)) if d != '.DS_Store']
  if len(data_folders) != num_classes:
    raise Exception(
      'Expected %d folders, one per class. Found %d instead.' % (
        num_classes, len(data_folders)))
  print(data_folders)
  return data_folders

train_folders = extract(train_filename)
test_folders = extract(test_filename)

Extracting data for notMNIST_large. This may take a while. Please wait.


Exception: Expected 4 folders, one per class. Found 10 instead.

##Problem1
Let's take a peek at some of the data to make sure it looks sensible. Each exemplar should be an image of a character A through J rendered in a different font. Display a sample of the images that we just downloaded. Hint: you can use the package IPython.display.

In [2]:
from IPython.display import display, Image
display(Image(filename="/home/tzeng/Repos/seaborn-practice/notMNIST_large/A/QmF1ZXJCb2RULVJlZ3Uub3Rm.png"))
display(Image(filename="/home/tzeng/Repos/seaborn-practice/notMNIST_large/B/a29ydW5pc2hpLnR0Zg==.png"))

In [3]:
root_l = '/home/tzeng/Repos/seaborn-practice/notMNIST_large/'
root_s = '/home/tzeng/Repos/seaborn-practice/notMNIST_small/'
classes = ['A', 'B', 'C', 'D']
train_folders = map(lambda d: os.path.join(root_l, d), classes)
test_folders = map(lambda d: os.path.join(root_s, d), classes)

print train_folders
print test_folders

['/home/tzeng/Repos/seaborn-practice/notMNIST_large/A', '/home/tzeng/Repos/seaborn-practice/notMNIST_large/B', '/home/tzeng/Repos/seaborn-practice/notMNIST_large/C', '/home/tzeng/Repos/seaborn-practice/notMNIST_large/D']
['/home/tzeng/Repos/seaborn-practice/notMNIST_small/A', '/home/tzeng/Repos/seaborn-practice/notMNIST_small/B', '/home/tzeng/Repos/seaborn-practice/notMNIST_small/C', '/home/tzeng/Repos/seaborn-practice/notMNIST_small/D']


In [4]:
import numpy as np

image_size = 28  # Pixel width and height.
pixel_depth = 255.0  # Number of levels per pixel.

def load(data_folders, min_num_images, max_num_images):
  dataset = np.ndarray(
    shape=(max_num_images, image_size, image_size), dtype=np.float32)
    # list with len(max_num_images), each is matrix with row=col=image_size
  labels = np.ndarray(shape=(max_num_images), dtype=np.int32)
  label_index = 0
  image_index = 0
  for folder in data_folders:
    print(folder)
    for image in os.listdir(folder):
      if image_index >= max_num_images:
        raise Exception('More images than expected: %d >= %d' % (
          image_index, max_num_images))
      image_file = os.path.join(folder, image)
      try:
        image_data = (ndimage.imread(image_file).astype(float) -
                      pixel_depth / 2) / pixel_depth
        # range of image data (0,255) -> (-0.5, 0.5)
        if image_data.shape != (image_size, image_size):
          raise Exception('Unexpected image shape: %s' % str(image_data.shape))
        dataset[image_index, :, :] = image_data
        labels[image_index] = label_index
        image_index += 1
      except IOError as e:
        print('Could not read:', image_file, ':', e, '- it\'s ok, skipping.')
    label_index += 1
  num_images = image_index
  dataset = dataset[0:num_images, :, :]
  labels = labels[0:num_images]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' % (
        num_images, min_num_images))
  print('Full dataset tensor:', dataset.shape)
  print('Mean:', np.mean(dataset))
  print('Standard deviation:', np.std(dataset))
  print('Labels:', labels.shape)
  return dataset, labels

train_dataset, train_labels = load(train_folders, 180000, 220000)
test_dataset, test_labels = load(test_folders, 7200, 8000)

/home/tzeng/Repos/seaborn-practice/notMNIST_large/A
('Could not read:', '/home/tzeng/Repos/seaborn-practice/notMNIST_large/A/RnJlaWdodERpc3BCb29rSXRhbGljLnR0Zg==.png', ':', IOError('cannot identify image file',), "- it's ok, skipping.")
('Could not read:', '/home/tzeng/Repos/seaborn-practice/notMNIST_large/A/SG90IE11c3RhcmQgQlROIFBvc3Rlci50dGY=.png', ':', IOError('cannot identify image file',), "- it's ok, skipping.")
('Could not read:', '/home/tzeng/Repos/seaborn-practice/notMNIST_large/A/Um9tYW5hIEJvbGQucGZi.png', ':', IOError('cannot identify image file',), "- it's ok, skipping.")
/home/tzeng/Repos/seaborn-practice/notMNIST_large/B
('Could not read:', '/home/tzeng/Repos/seaborn-practice/notMNIST_large/B/TmlraXNFRi1TZW1pQm9sZEl0YWxpYy5vdGY=.png', ':', IOError('cannot identify image file',), "- it's ok, skipping.")
/home/tzeng/Repos/seaborn-practice/notMNIST_large/C
/home/tzeng/Repos/seaborn-practice/notMNIST_large/D
('Could not read:', '/home/tzeng/Repos/seaborn-practice/notMNIST_lar

##Problem2
Let's verify that the data still looks good. Displaying a sample of the labels and images from the ndarray. Hint: you can use matplotlib.pyplot.

In [13]:
import matplotlib.pyplot as plt
#print train_dataset[0]
print train_labels[0:10]


[0 0 0 0 0 0 0 0 0 0]


In [5]:
# randomize the order of dataset
np.random.seed(133)
def randomize(dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_dataset, shuffled_labels

train_dataset, train_labels = randomize(train_dataset, train_labels)
test_dataset, test_labels = randomize(test_dataset, test_labels)

##Problem3
Convince yourself that the data is still good after shuffling!

In [6]:
train_dataset[0]

array([[-0.5       , -0.5       , -0.5       , -0.5       , -0.5       ,
        -0.5       , -0.5       , -0.5       , -0.49215686, -0.5       ,
        -0.0254902 ,  0.47647059,  0.43333334,  0.24509804, -0.14313726,
        -0.5       , -0.49215686, -0.5       , -0.5       , -0.5       ,
        -0.5       , -0.5       , -0.5       , -0.5       , -0.5       ,
        -0.5       , -0.5       , -0.5       ],
       [-0.5       , -0.5       , -0.5       , -0.5       , -0.5       ,
        -0.5       , -0.5       , -0.49607843, -0.5       , -0.41764706,
         0.42941177,  0.49607843,  0.5       ,  0.5       ,  0.5       ,
         0.04901961, -0.5       , -0.49215686, -0.5       , -0.5       ,
        -0.5       , -0.5       , -0.5       , -0.5       , -0.5       ,
        -0.5       , -0.5       , -0.5       ],
       [-0.5       , -0.5       , -0.5       , -0.5       , -0.5       ,
        -0.5       , -0.5       , -0.48823529, -0.49607843, -0.24509804,
         0.5       ,  0.5   

##Problem4
Another check: we expect the data to be balanced across classes. Verify that.

In [7]:
from collections import Counter
cnt_label = Counter()
for l in train_labels:
    cnt_label[l] += 1
print cnt_label

cnt_test_label = Counter()
for l in test_labels:
    cnt_test_label[l] += 1
print cnt_test_label

Counter({2: 52912, 1: 52911, 3: 52911, 0: 52909})
Counter({1: 1873, 2: 1873, 3: 1873, 0: 1872})


In [8]:
# create validation dataset
train_size = 100000
valid_size = 10000

valid_dataset = train_dataset[:valid_size,:,:]
valid_labels = train_labels[:valid_size]
sub_train_dataset = train_dataset[valid_size:valid_size+train_size,:,:]
sub_train_labels = train_labels[valid_size:valid_size+train_size]
print('Training', sub_train_dataset.shape, sub_train_labels.shape)
print('Validation', valid_dataset.shape, valid_labels.shape)

('Training', (100000, 28, 28), (100000,))
('Validation', (10000, 28, 28), (10000,))


In [9]:
import pickle

pickle_file = 'notMNIST.pickle'

try:
  f = open(pickle_file, 'wb')
  save = {
    'train_dataset': sub_train_dataset,
    'train_labels': sub_train_labels,
    'valid_dataset': valid_dataset,
    'valid_labels': valid_labels,
    'test_dataset': test_dataset,
    'test_labels': test_labels,
    }
  pickle.dump(save, f, pickle.HIGHEST_PROTOCOL)
  f.close()
except Exception as e:
  print('Unable to save data to', pickle_file, ':', e)
  raise

In [10]:
statinfo = os.stat(pickle_file)
print('Compressed pickle size:', statinfo.st_size)

('Compressed pickle size:', 368922254)


##Problem 5
By construction, this dataset might contain a lot of overlapping samples, including training data that's also contained in the validation and test set! Overlap between training and test can skew the results if you expect to use your model in an environment where there is never an overlap, but are actually ok if you expect to see training samples recur when you use it. Measure how much overlap there is between training, validation and test samples.

Optional questions:
- What about near duplicates between datasets? (images that are almost identical)
- Create a sanitized validation and test set, and compare your accuracy on those in subsequent assignments.


In [11]:
# reshape 28*28 matrix to 784*1

train_dataset_1 = np.ndarray(shape=(train_size, image_size*image_size), dtype=np.float32)
for ix, m in enumerate(sub_train_dataset):
    train_dataset_1[ix, :] = m.reshape(image_size*image_size,)
print train_dataset_1.shape


(100000, 784)


In [12]:
print sub_train_labels.shape
print sub_train_labels[0:10]

(100000,)
[0 3 1 2 2 3 2 3 3 2]


In [ ]:
"""
from scipy import stats

stats.pearsonr(train_dataset_1[0], train_dataset_1[1])

corr_dict = {}
for i in xrange(train_size-1):
    for j in xrange(i+1, train_size):
        corr_tuple = stats.pearsonr(train_dataset_1[i], train_dataset_1[j])
        if corr_tuple[0] == 1.0:
            print (i,j)
"""

In [13]:
print train_dataset_1[0], train_dataset_1[1]
print train_labels[0], train_labels[1]

[-0.5        -0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.48823529 -0.5        -0.28431374  0.0254902  -0.1627451  -0.47254902
 -0.5        -0.49215686 -0.5        -0.5        -0.5        -0.5        -0.5
 -0.5        -0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.5        -0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.48823529 -0.5        -0.21764706  0.46078432  0.5         0.5
  0.28431374 -0.34705883 -0.5        -0.49215686 -0.5        -0.5        -0.5
 -0.5        -0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.5        -0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.49215686 -0.5        -0.31568629  0.44901961  0.5         0.48039216
  0.48823529  0.5         0.42156863 -0.31960785 -0.5        -0.49215686
 -0.5        -0.5        -0.5        -0.5        -0.5        -0.5        -0.5
 -0.5        -0.5        

##Problem 6
Let's get an idea of what an off-the-shelf classifier can give you on this data. It's always good to check that there is something to learn, and that it's a problem that is not so trivial that a canned solution solves it.

Train a simple model on this data using 50, 100, 1000 and 5000 training samples. Hint: you can use the LogisticRegression model from sklearn.linear_model.

Optional question: train an off-the-shelf model on all the data!


In [15]:
from sklearn.linear_model import LogisticRegression

###First try with solver='newton-cg'

In [16]:
logreg = LogisticRegression(solver='newton-cg')
logreg.fit(train_dataset_1, sub_train_labels)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='newton-cg', tol=0.0001,
          verbose=0, warm_start=False)

In [19]:
print test_dataset.shape
test_dataset_1 =  np.ndarray(shape=(7491, image_size*image_size), dtype=np.float32)
for ix, m in enumerate(test_dataset):
    test_dataset_1[ix, :] = m.reshape(image_size*image_size,)
print test_dataset_1.shape
print test_labels.shape

(7491, 28, 28)
(7491, 784)
(7491,)


In [21]:
pred_output = logreg.predict(test_dataset_1)

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(test_labels, pred_output)

0.94927246028567613

In [25]:
logreg.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 1,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'newton-cg',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

###try with solver='lbfgs'

In [24]:
logreg2 = LogisticRegression(solver='lbfgs',n_jobs=4)
logreg2.fit(train_dataset_1, sub_train_labels)

pred_output2 = logreg2.predict(test_dataset_1)
print accuracy_score(test_labels, pred_output2)

0.949939927913


In [26]:
logreg2.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 4,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [33]:
logreg3 = LogisticRegression(C=0.5, solver='lbfgs',n_jobs=-1)
logreg3.fit(train_dataset_1, sub_train_labels)

pred_output3 = logreg3.predict(test_dataset_1)
print accuracy_score(test_labels, pred_output3)

0.949806434388


In [28]:
logreg3.get_params()

{'C': 0.5,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'max_iter': 100,
 'multi_class': 'ovr',
 'n_jobs': 4,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

In [32]:
print logreg3.intercept_
print logreg3.n_iter_

[-1.33968209 -2.0619984  -2.70262316 -2.07753372]
[100 100 100 100]


In [34]:
logreg4 = LogisticRegression(max_iter=200, solver='lbfgs',n_jobs=-1)
logreg4.fit(train_dataset_1, sub_train_labels)

pred_output4 = logreg4.predict(test_dataset_1)
print accuracy_score(test_labels, pred_output4)
print logreg4.intercept_
print logreg4.n_iter_

0.949539447337
[-1.35644493 -2.06403386 -2.71585575 -2.08151251]
[200 200 200 200]


In [35]:
import cPickle as pickle
loading = pickle.load(open('notMNIST.pickle'))
for key,val in loading.items():
    exec(key + '=val')

In [36]:
print train_dataset.shape
print valid_dataset.shape
print test_dataset.shape

(100000, 28, 28)
(10000, 28, 28)
(7491, 28, 28)
